In [17]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
import cvxpy as cp

In [18]:
df = pd.read_excel('data.xlsx', header=None)
requirements = df.values.flatten()
requirements

array([ 11,   5,   4,   7,  16,   6,   5,   7,  13,   6,   5,   7,  12,
         5,   4,   6,   9,   5,   5,  11,  29,  21,  17,  20,  27,  13,
         9,  10,  16,   6,   5,   7,  11,   5,   5,   6,  12,   7,   7,
        10,  15,  10,   9,  11,  15,  10,  10,  16,  26,  21,  23,  36,
        50,  45,  45,  49,  57,  43,  40,  44,  52,  43,  42,  45,  52,
        41,  39,  41,  48,  35,  34,  35,  42,  34,  36,  43,  55,  48,
        54,  65,  80,  70,  74,  85, 101,  89,  88,  90, 100,  87,  88,
        89, 104,  89,  89,  90, 106,  96,  94,  99, 109,  99,  96, 102],
      dtype=int64)

In [19]:
total_week = 104
waste_rate = 0.2
k = 10
requirements = requirements[0:total_week]

In [20]:
hand_new = cp.Variable(total_week, integer=True)
hand_work = cp.Parameter(total_week)
hand_work.value = 4 * requirements
hand_rest = cp.Variable(total_week, integer=True)
hand_teach = cp.Variable(total_week, integer=True)
hand_skill = cp.Variable(total_week, integer=True)

body_new = cp.Variable(total_week, integer=True)
body_work = cp.Parameter(total_week)
body_work.value = requirements
body_rest = cp.Variable(total_week, integer=True)
body_tested = cp.Variable(total_week, integer=True)
waste = cp.Parameter(total_week)
waste.value = np.round(waste_rate * 4 * requirements)

In [21]:
cons = [
    hand_skill == hand_rest + hand_teach + hand_work,
    body_tested == body_work + body_rest,
    k * hand_teach >= hand_new,
    
    hand_new >= 0,
    hand_rest >= 0,
    hand_teach >= 0,
    body_new >= 0,
    body_rest >= 0,

    hand_skill[0] == 50,
    body_tested[0] == 13
]

In [22]:
for t in range(total_week - 1):
    cons.append(hand_skill[t + 1] == hand_skill[t] - waste[t] + hand_new[t])
    cons.append(hand_rest[t + 1] >= hand_work[t] - waste[t])
    cons.append(body_tested[t + 1] == body_tested[t] + body_new[t])

In [23]:
obj = cp.Minimize(
    200 * cp.sum(body_new) + 
    100 * cp.sum(hand_new) + 
    5 * cp.sum(hand_rest) + 
    10 * cp.sum(body_rest) +
    10 * cp.sum(hand_new + hand_teach)
)

In [24]:
prob = cp.Problem(obj, cons)
prob.solve(solver=cp.MOSEK, verbose=True)

                                     CVXPY                                     
                                     v1.2.0                                    
(CVXPY) May 04 06:41:33 AM: Your problem has 728 variables, 319 constraints, and 312 parameters.
(CVXPY) May 04 06:41:33 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 04 06:41:33 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 04 06:41:33 AM: Compiling problem (target solver=MOSEK).
(CVXPY) May 04 06:41:33 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> MOSEK
(CVXPY) May 04 06:41:33 AM: Applying reduction Dcp2Cone
(CVXPY) May 04 06:41:33 AM: Applying reduction CvxAttr2Constr
(

524289.9999999999